In [2]:
import numpy as np 
import pandas as pd 
import sys
pd.set_option('display.max_colwidth', None)
import warnings
warnings.filterwarnings("ignore")

In [3]:
train = pd.read_csv('../dataset/application_train.csv')

In [4]:
bureau = pd.read_csv('../dataset/bureau.csv')

# HOW TO INTERPRET BUREAU DATA

This table talks about the Loan data of each unique customer with all financial institutions other than Home Credit
For each unique SK_ID_CURR we have multiple SK_ID_BUREAU Id's, each being a unique loan transaction from other financial institutions availed by the same customer and reported to the bureau. 

# EXAMPLE OF BUREAU TRANSACTIONS 

- In the example below customer with SK_ID_CURR = 100001 had  7 credit transactions before the current application. 

In [5]:
bureau[bureau['SK_ID_CURR'] == 100001]

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
248484,100001,5896630,Closed,currency 1,-857,0,-492.0,-553.0,NaN,0,112500.0,0.0,0.0,0.0,Consumer credit,-155,0.0
248485,100001,5896631,Closed,currency 1,-909,0,-179.0,-877.0,NaN,0,279720.0,0.0,0.0,0.0,Consumer credit,-155,0.0
248486,100001,5896632,Closed,currency 1,-879,0,-514.0,-544.0,NaN,0,91620.0,0.0,0.0,0.0,Consumer credit,-155,0.0
248487,100001,5896633,Closed,currency 1,-1572,0,-1329.0,-1328.0,NaN,0,85500.0,0.0,0.0,0.0,Consumer credit,-155,0.0
248488,100001,5896634,Active,currency 1,-559,0,902.0,NaN,NaN,0,337680.0,113166.0,0.0,0.0,Consumer credit,-6,4630.5
248489,100001,5896635,Active,currency 1,-49,0,1778.0,NaN,NaN,0,378000.0,373239.0,0.0,0.0,Consumer credit,-16,10822.5
248490,100001,5896636,Active,currency 1,-320,0,411.0,NaN,NaN,0,168345.0,110281.5,NaN,0.0,Consumer credit,-10,9364.5


# UNDERSTANDING OF VARIABLES 

CREDIT_ACTIVE - Current status of a Loan - Closed/ Active (2 values)

CREDIT_CURRENCY - Currency in which the transaction was executed -  Currency1, Currency2, Currency3, Currency4 
                                        ( 4 values)
                                        
CREDIT_DAY_OVERDUE -  Number of overdue days 

CREDIT_TYPE -  Consumer Credit, Credit card, Mortgage, Car loan, Microloan, Loan for working capital replemishment, 
                             Loan for Business development, Real estate loan, Unkown type of laon, Another type of loan. 
                             Cash loan, Loan for the purchase of equipment, Mobile operator loan, Interbank credit, 
                             Loan for purchase of shares ( 15 values )

DAYS_CREDIT -   Number of days ELAPSED since customer applied for CB credit with respect to current application 
Interpretation - Are these loans evenly spaced time intervals? Are they concentrated within a same time frame?


DAYS_CREDIT_ENDDATE - Number of days the customer CREDIT is valid at the time of application 

CREDIT_DAY_OVERDUE - Number of days the customer CREDIT is past the end date at the time of application

AMT_CREDIT_SUM -  Total available credit for a customer 

AMT_CREDIT_SUM_DEBT -  Total amount yet to be repayed

AMT_CREDIT_SUM_LIMIT -   Current Credit that has been utilized 

AMT_CREDIT_SUM_OVERDUE - Current credit payment that is overdue 

CNT_CREDIT_PROLONG - How many times was the Credit date prolonged 


# NOTE: 
For a given loan transaction 
 'AMT_CREDIT_SUM' =  'AMT_CREDIT_SUM_DEBT' +'AMT_CREDIT_SUM_LIMIT'



AMT_ANNUITY -  Annuity of the Credit Bureau data

DAYS_CREDIT_UPDATE -  Number of days before current application when last CREDIT UPDATE was received 

DAYS_ENDDATE_FACT -    Days since CB credit ended at the time of application 

AMT_CREDIT_MAX_OVERDUE - Maximum Credit amount overdue at the time of application 


# FEATURE ENGINEERING WITH BUREAU CREDIT 

# FEATURE 1 - NUMBER OF PAST LOANS PER CUSTOMER 

In [16]:
B = bureau[0:10000]
grp = B[['SK_ID_CURR', 'DAYS_CREDIT']].groupby(by = ['SK_ID_CURR'])['DAYS_CREDIT'].count().reset_index().rename(index=str, columns={'DAYS_CREDIT': 'BUREAU_LOAN_COUNT'})
B = B.merge(grp, on = ['SK_ID_CURR'], how = 'left')
print(B.shape)

(10000, 18)


In [17]:
grp

,SK_ID_CURR,BUREAU_LOAN_COUNT
0,100053,7
1,100568,7
2,100653,7
3,100802,1
4,100819,18
...,...,...
2006,455171,7
2007,455317,14
2008,456047,3
2009,456062,1


In [22]:
B

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,BUREAU_LOAN_COUNT
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,7
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN,7
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN,7
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN,7
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,386412,5726627,Closed,currency 1,-1604,0,-1300.0,-1316.0,NaN,0,27072.0,0.0,NaN,0.0,Consumer credit,-1298,0.0,5
9996,386412,5726628,Closed,currency 1,-1158,0,-975.0,-975.0,NaN,0,26284.5,0.0,NaN,0.0,Consumer credit,-969,0.0,5
9997,386412,5726629,Active,currency 1,-679,0,30948.0,NaN,NaN,0,42750.0,0.0,NaN,0.0,Credit card,-10,0.0,5
9998,215641,5726630,Closed,currency 1,-965,0,-600.0,-604.0,NaN,0,69750.0,0.0,0.0,0.0,Consumer credit,-602,10287.0,2


# FEATURE 2 - NUMBER OF TYPES OF PAST LOANS PER CUSTOMER 

In [23]:
B = bureau[0:10000]
grp = B[['SK_ID_CURR', 'CREDIT_TYPE']].groupby(by = ['SK_ID_CURR'])['CREDIT_TYPE'].nunique().reset_index().rename(index=str, columns={'CREDIT_TYPE': 'BUREAU_LOAN_TYPES'})
B = B.merge(grp, on = ['SK_ID_CURR'], how = 'left')
print(B.shape)

(10000, 18)


In [24]:
grp

,SK_ID_CURR,BUREAU_LOAN_TYPES
0,100053,2
1,100568,2
2,100653,2
3,100802,1
4,100819,2
...,...,...
2006,455171,2
2007,455317,2
2008,456047,2
2009,456062,1


# FEATURE 3 - AVERAGE NUMBER OF PAST LOANS PER TYPE PER CUSTOMER

# Is the Customer diversified in taking multiple types of Loan or Focused on a single type of loan


In [25]:
B = bureau[0:10000]
# Number of Loans per Customer
grp = B[['SK_ID_CURR', 'DAYS_CREDIT']].groupby(by = ['SK_ID_CURR'])['DAYS_CREDIT'].count().reset_index().rename(index=str, columns={'DAYS_CREDIT': 'BUREAU_LOAN_COUNT'})
B = B.merge(grp, on = ['SK_ID_CURR'], how = 'left')

# Number of types of Credit loans for each Customer 
grp = B[['SK_ID_CURR', 'CREDIT_TYPE']].groupby(by = ['SK_ID_CURR'])['CREDIT_TYPE'].nunique().reset_index().rename(index=str, columns={'CREDIT_TYPE': 'BUREAU_LOAN_TYPES'})
B = B.merge(grp, on = ['SK_ID_CURR'], how = 'left')

# Average Number of Loans per Loan Type
B['AVERAGE_LOAN_TYPE'] = B['BUREAU_LOAN_COUNT']/B['BUREAU_LOAN_TYPES']
del B['BUREAU_LOAN_COUNT'], B['BUREAU_LOAN_TYPES']
import gc
gc.collect()
print(B.shape)

(10000, 18)


In [28]:
B

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,AVERAGE_LOAN_TYPE
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,3.5
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN,3.5
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN,3.5
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN,3.5
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,386412,5726627,Closed,currency 1,-1604,0,-1300.0,-1316.0,NaN,0,27072.0,0.0,NaN,0.0,Consumer credit,-1298,0.0,2.5
9996,386412,5726628,Closed,currency 1,-1158,0,-975.0,-975.0,NaN,0,26284.5,0.0,NaN,0.0,Consumer credit,-969,0.0,2.5
9997,386412,5726629,Active,currency 1,-679,0,30948.0,NaN,NaN,0,42750.0,0.0,NaN,0.0,Credit card,-10,0.0,2.5
9998,215641,5726630,Closed,currency 1,-965,0,-600.0,-604.0,NaN,0,69750.0,0.0,0.0,0.0,Consumer credit,-602,10287.0,2.0


# FEATURE 4 - % OF ACTIVE LOANS FROM BUREAU DATA 

In [29]:
B = bureau[0:10000]
# Create a new dummy column for whether CREDIT is ACTIVE OR CLOED 
B['CREDIT_ACTIVE_BINARY'] = B['CREDIT_ACTIVE']

def f(x):
    if x == 'Closed':
        y = 0
    else:
        y = 1    
    return y

B['CREDIT_ACTIVE_BINARY'] = B.apply(lambda x: f(x.CREDIT_ACTIVE), axis = 1)

# Calculate mean number of loans that are ACTIVE per CUSTOMER 
grp = B.groupby(by = ['SK_ID_CURR'])['CREDIT_ACTIVE_BINARY'].mean().reset_index().rename(index=str, columns={'CREDIT_ACTIVE_BINARY': 'ACTIVE_LOANS_PERCENTAGE'})
B = B.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del B['CREDIT_ACTIVE_BINARY']
import gc
gc.collect()
print(B.shape)

B[B['SK_ID_CURR'] == 100653]

(10000, 18)


,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,ACTIVE_LOANS_PERCENTAGE
9688,100653,5726282,Active,currency 1,-256,0,109.0,NaN,NaN,0,97244.91,39081.42,NaN,0.0,Consumer credit,-37,0.0,0.571429
9689,100653,5726283,Closed,currency 1,-2474,0,-2260.0,-2289.0,NaN,0,30262.14,0.00,NaN,0.0,Consumer credit,-856,NaN,0.571429
9690,100653,5726284,Closed,currency 1,-2258,0,-1924.0,-1924.0,NaN,0,47156.85,0.00,NaN,0.0,Consumer credit,-856,0.0,0.571429
9691,100653,5726285,Closed,currency 1,-1677,0,-946.0,-946.0,NaN,0,71068.50,0.00,0.0,0.0,Consumer credit,-895,0.0,0.571429
9692,100653,5726286,Active,currency 1,-182,0,-152.0,NaN,NaN,0,225000.00,215923.50,0.0,0.0,Credit card,-82,0.0,0.571429
9693,100653,5726287,Active,currency 1,-1564,0,NaN,NaN,NaN,0,87750.00,0.00,NaN,0.0,Credit card,-1006,0.0,0.571429
9694,100653,5726288,Active,currency 1,-252,0,3401.0,NaN,NaN,0,315000.00,201766.50,NaN,0.0,Credit card,-22,NaN,0.571429


# FEATURE 5

# AVERAGE NUMBER OF DAYS BETWEEN SUCCESSIVE PAST APPLICATIONS FOR EACH CUSTOMER 

# How often did the customer take credit in the past? Was it spaced out at regular time intervals - a signal of good financial planning OR were the loans concentrated around a smaller time frame - indicating potential financial trouble?


In [30]:
B = bureau[0:10000]
# Groupby each Customer and Sort values of DAYS_CREDIT in ascending order
grp = B[['SK_ID_CURR', 'SK_ID_BUREAU', 'DAYS_CREDIT']].groupby(by = ['SK_ID_CURR'])
grp1 = grp.apply(lambda x: x.sort_values(['DAYS_CREDIT'], ascending = False)).reset_index(drop = True)#rename(index = str, columns = {'DAYS_CREDIT': 'DAYS_CREDIT_DIFF'})
print("Grouping and Sorting done")

# Calculate Difference between the number of Days 
grp1['DAYS_CREDIT1'] = grp1['DAYS_CREDIT']*-1
grp1['DAYS_DIFF'] = grp1.groupby(by = ['SK_ID_CURR'])['DAYS_CREDIT1'].diff()
grp1['DAYS_DIFF'] = grp1['DAYS_DIFF'].fillna(0).astype('uint32')
del grp1['DAYS_CREDIT1'], grp1['DAYS_CREDIT'], grp1['SK_ID_CURR']
gc.collect()
print("Difference days calculated")

B = B.merge(grp1, on = ['SK_ID_BUREAU'], how = 'left')
print("Difference in Dates between Previous CB applications is CALCULATED ")
print(B.shape)

Grouping and Sorting done
Difference days calculated
Difference in Dates between Previous CB applications is CALCULATED 
(10000, 18)


In [31]:
B

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,DAYS_DIFF
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,224
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN,5
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN,160
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN,0
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN,132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,386412,5726627,Closed,currency 1,-1604,0,-1300.0,-1316.0,NaN,0,27072.0,0.0,NaN,0.0,Consumer credit,-1298,0.0,445
9996,386412,5726628,Closed,currency 1,-1158,0,-975.0,-975.0,NaN,0,26284.5,0.0,NaN,0.0,Consumer credit,-969,0.0,200
9997,386412,5726629,Active,currency 1,-679,0,30948.0,NaN,NaN,0,42750.0,0.0,NaN,0.0,Credit card,-10,0.0,0
9998,215641,5726630,Closed,currency 1,-965,0,-600.0,-604.0,NaN,0,69750.0,0.0,0.0,0.0,Consumer credit,-602,10287.0,414


# FEATURE 6  

# % of LOANS PER CUSTOMER WHERE END DATE FOR CREDIT IS PAST

 # INTERPRETING CREDIT_DAYS_ENDDATE 
 
 #  NEGATIVE VALUE - Credit date was in the past at time of application( Potential Red Flag !!! )
 
 # POSITIVE VALUE - Credit date is in the future at time of application ( Potential Good Sign !!!!)
 
 # NOTE : This is not the same as % of Active loans since Active loans 
 # can have Negative and Positive values for DAYS_CREDIT_ENDDATE

In [45]:
B = bureau[0:10000]
B['CREDIT_ENDDATE_BINARY'] = B['DAYS_CREDIT_ENDDATE']

def f(x):
    if x<0:
        y = 0
    else:
        y = 1   
    return y

B['CREDIT_ENDDATE_BINARY'] = B.apply(lambda x: f(x.DAYS_CREDIT_ENDDATE), axis = 1)
print("New Binary Column calculated")

grp = B.groupby(by = ['SK_ID_CURR'])['CREDIT_ENDDATE_BINARY'].mean().reset_index().rename(index=str, columns={'CREDIT_ENDDATE_BINARY': 'CREDIT_ENDDATE_PERCENTAGE'})
B = B.merge(grp, on = ['SK_ID_CURR'], how = 'left')

del B['CREDIT_ENDDATE_BINARY']
gc.collect()
print(B.shape)

New Binary Column calculated
(10000, 18)


In [46]:
B

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,CREDIT_ENDDATE_PERCENTAGE
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,0.857143
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN,0.857143
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,0.0,NaN,0.0,Consumer credit,-16,NaN,0.857143
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,0.0,NaN,0.0,Credit card,-16,NaN,0.857143
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,0.0,NaN,0.0,Consumer credit,-21,NaN,0.857143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,386412,5726627,Closed,currency 1,-1604,0,-1300.0,-1316.0,NaN,0,27072.0,0.0,NaN,0.0,Consumer credit,-1298,0.0,0.200000
9996,386412,5726628,Closed,currency 1,-1158,0,-975.0,-975.0,NaN,0,26284.5,0.0,NaN,0.0,Consumer credit,-969,0.0,0.200000
9997,386412,5726629,Active,currency 1,-679,0,30948.0,NaN,NaN,0,42750.0,0.0,NaN,0.0,Credit card,-10,0.0,0.200000
9998,215641,5726630,Closed,currency 1,-965,0,-600.0,-604.0,NaN,0,69750.0,0.0,0.0,0.0,Consumer credit,-602,10287.0,0.000000


# FEATURE 7 

# AVERAGE NUMBER OF DAYS IN WHICH CREDIT EXPIRES IN FUTURE -INDICATION OF CUSTOMER DELINQUENCY IN FUTURE??

In [33]:
# Repeating Feature 6 to Calculate all transactions with ENDATE as POSITIVE VALUES 

B = bureau[0:10000]
# Dummy column to calculate 1 or 0 values. 1 for Positive CREDIT_ENDDATE and 0 for Negative
B['CREDIT_ENDDATE_BINARY'] = B['DAYS_CREDIT_ENDDATE']

def f(x):
    if x<0:
        y = 0
    else:
        y = 1   
    return y

B['CREDIT_ENDDATE_BINARY'] = B.apply(lambda x: f(x.DAYS_CREDIT_ENDDATE), axis = 1)
print("New Binary Column calculated")

# We take only positive values of  ENDDATE since we are looking at Bureau Credit VALID IN FUTURE 
# as of the date of the customer's loan application with Home Credit 
B1 = B[B['CREDIT_ENDDATE_BINARY'] == 1]
B1.shape

#Calculate Difference in successive future end dates of CREDIT 

# Create Dummy Column for CREDIT_ENDDATE 
B1['DAYS_CREDIT_ENDDATE1'] = B1['DAYS_CREDIT_ENDDATE']
# Groupby Each Customer ID 
grp = B1[['SK_ID_CURR', 'SK_ID_BUREAU', 'DAYS_CREDIT_ENDDATE1']].groupby(by = ['SK_ID_CURR'])
# Sort the values of CREDIT_ENDDATE for each customer ID 
grp1 = grp.apply(lambda x: x.sort_values(['DAYS_CREDIT_ENDDATE1'], ascending = True)).reset_index(drop = True)
del grp
gc.collect()
print("Grouping and Sorting done")

# Calculate the Difference in ENDDATES and fill missing values with zero 
grp1['DAYS_ENDDATE_DIFF'] = grp1.groupby(by = ['SK_ID_CURR'])['DAYS_CREDIT_ENDDATE1'].diff()
grp1['DAYS_ENDDATE_DIFF'] = grp1['DAYS_ENDDATE_DIFF'].fillna(0).astype('uint32')
del grp1['DAYS_CREDIT_ENDDATE1'], grp1['SK_ID_CURR']
gc.collect()
print("Difference days calculated")

# Merge new feature 'DAYS_ENDDATE_DIFF' with original Data frame for BUREAU DATA
B = B.merge(grp1, on = ['SK_ID_BUREAU'], how = 'left')
del grp1
gc.collect()

# Calculate Average of DAYS_ENDDATE_DIFF

grp = B[['SK_ID_CURR', 'DAYS_ENDDATE_DIFF']].groupby(by = ['SK_ID_CURR'])['DAYS_ENDDATE_DIFF'].mean().reset_index().rename( index = str, columns = {'DAYS_ENDDATE_DIFF': 'AVG_ENDDATE_FUTURE'})
B = B.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
#del B['DAYS_ENDDATE_DIFF']
del B['CREDIT_ENDDATE_BINARY'], B['DAYS_CREDIT_ENDDATE']
gc.collect()
print(B.shape)

New Binary Column calculated
Grouping and Sorting done
Difference days calculated
(10000, 18)


In [34]:
# Verification of Feature 
B[B['SK_ID_CURR'] == 100653]
# In the Data frame below we have 3 values not NAN 
# Average of 3 values = (0 +0 + 3292)/3 = 1097.33 
#The NAN Values are Not Considered since these values DO NOT HAVE A FUTURE CREDIT END DATE 

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,DAYS_ENDDATE_DIFF,AVG_ENDDATE_FUTURE
9688,100653,5726282,Active,currency 1,-256,0,NaN,NaN,0,97244.91,39081.42,NaN,0.0,Consumer credit,-37,0.0,0.0,1097.333333
9689,100653,5726283,Closed,currency 1,-2474,0,-2289.0,NaN,0,30262.14,0.00,NaN,0.0,Consumer credit,-856,NaN,NaN,1097.333333
9690,100653,5726284,Closed,currency 1,-2258,0,-1924.0,NaN,0,47156.85,0.00,NaN,0.0,Consumer credit,-856,0.0,NaN,1097.333333
9691,100653,5726285,Closed,currency 1,-1677,0,-946.0,NaN,0,71068.50,0.00,0.0,0.0,Consumer credit,-895,0.0,NaN,1097.333333
9692,100653,5726286,Active,currency 1,-182,0,NaN,NaN,0,225000.00,215923.50,0.0,0.0,Credit card,-82,0.0,NaN,1097.333333
9693,100653,5726287,Active,currency 1,-1564,0,NaN,NaN,0,87750.00,0.00,NaN,0.0,Credit card,-1006,0.0,0.0,1097.333333
9694,100653,5726288,Active,currency 1,-252,0,NaN,NaN,0,315000.00,201766.50,NaN,0.0,Credit card,-22,NaN,3292.0,1097.333333


# FEATURE 8 - DEBT OVER CREDIT RATIO 
# The Ratio of Total Debt to Total Credit for each Customer 
# A High value may be a red flag indicative of potential default

In [38]:
B[~B['AMT_CREDIT_SUM_LIMIT'].isnull()][0:2]

# WE can see in the Table Below 
# AMT_CREDIT_SUM = AMT_CREDIT_SUM_DEBT + AMT_CREDIT_SUM_LIMIT

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,OVERDUE_DEBT_RATIO
5,215354,5714467,Active,currency 1,-273,0,27460.0,NaN,0.0,0,180000.0,71017.38,108982.62,0.0,Credit card,-31,NaN,0.0
6,215354,5714468,Active,currency 1,-43,0,79.0,NaN,0.0,0,42103.8,42103.80,0.00,0.0,Consumer credit,-22,NaN,0.0


In [39]:
B = bureau[0:10000]

B['AMT_CREDIT_SUM_DEBT'] = B['AMT_CREDIT_SUM_DEBT'].fillna(0)
B['AMT_CREDIT_SUM'] = B['AMT_CREDIT_SUM'].fillna(0)

grp1 = B[['SK_ID_CURR', 'AMT_CREDIT_SUM_DEBT']].groupby(by = ['SK_ID_CURR'])['AMT_CREDIT_SUM_DEBT'].sum().reset_index().rename( index = str, columns = { 'AMT_CREDIT_SUM_DEBT': 'TOTAL_CUSTOMER_DEBT'})
grp2 = B[['SK_ID_CURR', 'AMT_CREDIT_SUM']].groupby(by = ['SK_ID_CURR'])['AMT_CREDIT_SUM'].sum().reset_index().rename( index = str, columns = { 'AMT_CREDIT_SUM': 'TOTAL_CUSTOMER_CREDIT'})

B = B.merge(grp1, on = ['SK_ID_CURR'], how = 'left')
B = B.merge(grp2, on = ['SK_ID_CURR'], how = 'left')
del grp1, grp2
gc.collect()

B['DEBT_CREDIT_RATIO'] = B['TOTAL_CUSTOMER_DEBT']/B['TOTAL_CUSTOMER_CREDIT']

del B['TOTAL_CUSTOMER_DEBT'], B['TOTAL_CUSTOMER_CREDIT']
gc.collect()
print(B.shape)

(10000, 18)


In [40]:
B

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,DEBT_CREDIT_RATIO
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,0.075002
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN,0.075002
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,0.0,NaN,0.0,Consumer credit,-16,NaN,0.075002
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,0.0,NaN,0.0,Credit card,-16,NaN,0.075002
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,0.0,NaN,0.0,Consumer credit,-21,NaN,0.075002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,386412,5726627,Closed,currency 1,-1604,0,-1300.0,-1316.0,NaN,0,27072.0,0.0,NaN,0.0,Consumer credit,-1298,0.0,0.000000
9996,386412,5726628,Closed,currency 1,-1158,0,-975.0,-975.0,NaN,0,26284.5,0.0,NaN,0.0,Consumer credit,-969,0.0,0.000000
9997,386412,5726629,Active,currency 1,-679,0,30948.0,NaN,NaN,0,42750.0,0.0,NaN,0.0,Credit card,-10,0.0,0.000000
9998,215641,5726630,Closed,currency 1,-965,0,-600.0,-604.0,NaN,0,69750.0,0.0,0.0,0.0,Consumer credit,-602,10287.0,0.000000


# FEATURE 9 - OVERDUE OVER DEBT RATIO 
#  What fraction of total Debt is overdue per customer?
# A high value could indicate a potential DEFAULT 

In [41]:
B = bureau[0:10000]

B['AMT_CREDIT_SUM_DEBT'] = B['AMT_CREDIT_SUM_DEBT'].fillna(0)
B['AMT_CREDIT_SUM_OVERDUE'] = B['AMT_CREDIT_SUM_OVERDUE'].fillna(0)

grp1 = B[['SK_ID_CURR', 'AMT_CREDIT_SUM_DEBT']].groupby(by = ['SK_ID_CURR'])['AMT_CREDIT_SUM_DEBT'].sum().reset_index().rename( index = str, columns = { 'AMT_CREDIT_SUM_DEBT': 'TOTAL_CUSTOMER_DEBT'})
grp2 = B[['SK_ID_CURR', 'AMT_CREDIT_SUM_OVERDUE']].groupby(by = ['SK_ID_CURR'])['AMT_CREDIT_SUM_OVERDUE'].sum().reset_index().rename( index = str, columns = { 'AMT_CREDIT_SUM_OVERDUE': 'TOTAL_CUSTOMER_OVERDUE'})

B = B.merge(grp1, on = ['SK_ID_CURR'], how = 'left')
B = B.merge(grp2, on = ['SK_ID_CURR'], how = 'left')
del grp1, grp2
gc.collect()

B['OVERDUE_DEBT_RATIO'] = B['TOTAL_CUSTOMER_OVERDUE']/B['TOTAL_CUSTOMER_DEBT']

del B['TOTAL_CUSTOMER_OVERDUE'], B['TOTAL_CUSTOMER_DEBT']
gc.collect()
print(B.shape)

(10000, 18)


In [42]:
B

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,OVERDUE_DEBT_RATIO
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,0.0
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN,0.0
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,0.0,NaN,0.0,Consumer credit,-16,NaN,0.0
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,0.0,NaN,0.0,Credit card,-16,NaN,0.0
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,0.0,NaN,0.0,Consumer credit,-21,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,386412,5726627,Closed,currency 1,-1604,0,-1300.0,-1316.0,NaN,0,27072.0,0.0,NaN,0.0,Consumer credit,-1298,0.0,NaN
9996,386412,5726628,Closed,currency 1,-1158,0,-975.0,-975.0,NaN,0,26284.5,0.0,NaN,0.0,Consumer credit,-969,0.0,NaN
9997,386412,5726629,Active,currency 1,-679,0,30948.0,NaN,NaN,0,42750.0,0.0,NaN,0.0,Credit card,-10,0.0,NaN
9998,215641,5726630,Closed,currency 1,-965,0,-600.0,-604.0,NaN,0,69750.0,0.0,0.0,0.0,Consumer credit,-602,10287.0,NaN


# FEATURE 10 - AVERAGE NUMBER OF LOANS PROLONGED 

In [43]:
B = bureau[0:10000]

B['CNT_CREDIT_PROLONG'] = B['CNT_CREDIT_PROLONG'].fillna(0)
grp = B[['SK_ID_CURR', 'CNT_CREDIT_PROLONG']].groupby(by = ['SK_ID_CURR'])['CNT_CREDIT_PROLONG'].mean().reset_index().rename( index = str, columns = { 'CNT_CREDIT_PROLONG': 'AVG_CREDITDAYS_PROLONGED'})
B = B.merge(grp, on = ['SK_ID_CURR'], how = 'left')
print(B.shape)

(10000, 18)


In [44]:
B

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,AVG_CREDITDAYS_PROLONGED
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,0.0
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN,0.0
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,0.0,NaN,0.0,Consumer credit,-16,NaN,0.0
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,0.0,NaN,0.0,Credit card,-16,NaN,0.0
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,0.0,NaN,0.0,Consumer credit,-21,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,386412,5726627,Closed,currency 1,-1604,0,-1300.0,-1316.0,NaN,0,27072.0,0.0,NaN,0.0,Consumer credit,-1298,0.0,0.0
9996,386412,5726628,Closed,currency 1,-1158,0,-975.0,-975.0,NaN,0,26284.5,0.0,NaN,0.0,Consumer credit,-969,0.0,0.0
9997,386412,5726629,Active,currency 1,-679,0,30948.0,NaN,NaN,0,42750.0,0.0,NaN,0.0,Credit card,-10,0.0,0.0
9998,215641,5726630,Closed,currency 1,-965,0,-600.0,-604.0,NaN,0,69750.0,0.0,0.0,0.0,Consumer credit,-602,10287.0,0.0
